In [ ]:
## Import

import paho.mqtt.client as mqtt
import requests
from json import dumps as dumps_json, loads as loads_json
from datetime import datetime
import re 

## Deklarace

SERVER = '147.228.124.230'  # RPi IP adress
TOPIC = 'ite/blue' # Team Blue

url_base = 'https://uvb1bb4153.execute-api.eu-central-1.amazonaws.com/Prod'
body_login = {'username': 'Blue', 'password': 'n96{ZYV7'}
headers_base_login = {'Content-Type': 'application/json'}

## Login

def login(body_login):
    url_login = url_base+'/login'
    
    login_data = loads_json(requests.post(url_login, data=dumps_json(body_login), headers=headers_base_login).text)
    print('\nLogin data:', login_data)
    teamUUID = login_data['teamUUID']
    print('\nteamUUID:', teamUUID)
    return teamUUID
#teamUUID = 'f32c6941-bc2d-41b2-8bb3-cb6082427613' #blue

## Get sensors

def get_sensors(teamUUID):
    url_sensors = url_base+'/sensors'
    headers_sensors = dict(headers_base_login)
    headers_sensors.update({'teamUUID': teamUUID})

    sensors_data = loads_json(requests.get(url_sensors, headers=headers_sensors).text)
    print('\nSensors data:', sensors_data)
    return sensors_data
#sensorUUIDblue = 'd384a529-6227-4133-afc9-4f5a16665f1f'

## Spojeni s RPi

# Pripojeni k RPi serveru
def on_connect(client, userdata, mid, qos):
    print('Connected with result code qos:', str(qos))
    
    client.subscribe(TOPIC) #subscribenuti topicu 

# Ziskani message ze serveru
def on_message(client, userdata, msg):
    global teamUUID
    global sensorUUID
    
    if (msg.payload == 'Q'):
        client.disconnect()
    message = msg
    print(msg.topic, msg.qos, msg.payload)
    
    mes = str(message.payload)
    mes_dict, time = regex(mes)
    store = store_meas(teamUUID, sensorUUID, mes_dict, time)


## Zpracovani zpravy

def regex(mes):
    source = re.search("(source){1}", mes).group().strip()
    team_name = re.search("(team_name){1}", mes).group().strip()
    created_on = re.search("(created_on){1}", mes).group().strip()
    temperature = re.search("(temperature){1}", mes).group().strip()
    #print(source + ", " + team_name + ", " + created_on + ", " + temperature)

    value1 = re.search('(?<="source": ").+(?=", "team_name")', mes).group().strip() # "fake"/"real"
    value2 = re.search('(?<="team_name": ").+(?=", "created_on")', mes).group().strip() # barva tymu
    value3 = re.search('(?<="created_on": ").+(?=", "temperature")', mes).group().strip()
    value4 = re.search('(?<="temperature": ).+(?=})', mes).group().strip()
    #print(value1 + ", " + value2 + ", " + value3 + ", " + value4)

    mes_dict = {source: value1, team_name: value2, created_on: value3, temperature: float(value4)}

    tms = datetime.strptime(mes_dict['created_on'], '%Y-%m-%dT%H:%M:%S.%f')
    time = tms.strftime("%Y-%m-%d") + 'T'+ tms.strftime("%H:%M:%S.") + str(int(tms.strftime("%f"))//1000) + '+' + '01:00'
    return mes_dict, time


## Store measurement
def store_meas(teamUUID, sensorUUID, mes_dict, time): 
    url_measurement = url_base+'/measurements'
    headers_base_measurement = {'Content-Type': 'application/json', 'teamUUID': teamUUID }
    
    body_measurement = {'createdOn': time, 'sensorUUID': sensorUUID, 'temperature': str(round(mes_dict['temperature'],1)), 'status': 'TEST'}
    F = requests.post(url_measurement, data=dumps_json(body_measurement), headers=headers_base_measurement)
    return F


# Main - pouziti standardu MQTT

if __name__ == '__main__':

    teamUUID = login(body_login)

    sensor = get_sensors(teamUUID)
    sensorUUID = sensor[0]['sensorUUID']
    
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message

    client.username_pw_set('mqtt_student', password='pivo')

    client.connect(SERVER, 1883, 60)

    client.loop_forever()
  
    #mes = str(message.payload)
    #mes = b'{"source": "fake", "team_name": "blue", "created_on": "2020-04-28T20:48:54.850744", "temperature": 22.200536974016668}'
    #mes = str(b'{"source": "fake", "team_name": "blue", "created_on": "2020-04-28T20:48:54.850744", "temperature": 22.200536974016668}')
    
